In [1]:
client_id = 
client_secret = 

import pandas as pd
from datetime import datetime, timedelta
import requests
import re
def handle_authorization(data):
    
    access_token = data["access_token"]
    print(f" {access_token}")
    return access_token
    
def get_twitch_authorization(client_id, client_secret):
    url = f"https://id.twitch.tv/oauth2/token"
    params = {
        "client_id": client_id,
        "client_secret": client_secret,
        "grant_type": "client_credentials",
    }
    
    response = requests.post(url, params=params)
    data = response.json()
    
    handle_authorization(data)



# Obtén la autorización de Twitch
get_twitch_authorization(client_id, client_secret)

 rcfd1anvwovhforvy8oqsp08mvat5j


In [2]:
oauth_token = 

In [3]:
def extract_channel_name(url):
    pattern = r"https?:\/\/(?:www\.|go\.)?twitch\.tv\/([a-zA-Z0-9_]{4,25})"
    match = re.search(pattern, url)
    if match:
        return match.group(1)
    return None
def get_streamer_data(client_id, url):
    channel_name = extract_channel_name(url)
    if not channel_name:
        return None
    
    url = f"https://api.twitch.tv/helix/users"
    
    params = {
        "login": channel_name
    }
    headers = {
        "Client-ID": client_id,
        "Authorization": f"Bearer {oauth_token}"
    }
    
    response = requests.get(url, params=params, headers=headers)
    data = response.json()
    print(data)
    if "data" in data and len(data["data"]) > 0:
        user_id = data["data"][0]["id"]
        stream_url = f"https://api.twitch.tv/helix/streams?user_id={user_id}"
        response = requests.get(stream_url, headers=headers)
        stream_data = response.json()
        
        if "data" in stream_data and len(stream_data["data"]) > 0:
            return stream_data["data"][0]
    
    return None
url = "https://www.twitch.tv/juansguarnizo"
streamer_data = get_streamer_data(client_id, url)

if streamer_data:
    data = f"Datos del streamer {channel_url}:"

{'data': [{'id': '121510236', 'login': 'juansguarnizo', 'display_name': 'juansguarnizo', 'type': '', 'broadcaster_type': 'partner', 'description': 'Fenómeno viral 2020, 2021, 2022 y 2023. Presidente de Aniquiladores FC. Ganador de un premio Esland, un premio Eliot, 4 premios Simones, 14 premios Zorman y voz de Spectacular Spider-Man en el cine.', 'profile_image_url': 'https://static-cdn.jtvnw.net/jtv_user_pictures/74586414-e27b-4347-89c5-109e42ac3e1d-profile_image-300x300.png', 'offline_image_url': 'https://static-cdn.jtvnw.net/jtv_user_pictures/d66a5b94-2d38-4d55-81e3-f6e9bf5b3d65-channel_offline_image-1920x1080.png', 'view_count': 0, 'created_at': '2016-04-11T03:34:11Z'}]}


In [7]:
data = {'data': [{'id': '121510236', 'login': 'juansguarnizo', 'display_name': 'juansguarnizo', 'type': '', 'broadcaster_type': 'partner', 'description': 'Fenómeno viral 2020, 2021, 2022 y 2023. Presidente de Aniquiladores FC. Ganador de un premio Esland, un premio Eliot, 4 premios Simones, 14 premios Zorman y voz de Spectacular Spider-Man en el cine.', 'profile_image_url': 'https://static-cdn.jtvnw.net/jtv_user_pictures/74586414-e27b-4347-89c5-109e42ac3e1d-profile_image-300x300.png', 'offline_image_url': 'https://static-cdn.jtvnw.net/jtv_user_pictures/d66a5b94-2d38-4d55-81e3-f6e9bf5b3d65-channel_offline_image-1920x1080.png', 'view_count': 0, 'created_at': '2016-04-11T03:34:11Z'}]}
data_dict = data['data'][0]
id_canal =  list(data_dict.values())[0]
id_canal

'121510236'

In [8]:
def obtener_vods(id_canal):
    # Calcular la fecha hace un año desde hoy
    fecha_inicio = datetime.now() - timedelta(days=730)
    fecha_inicio_str = fecha_inicio.strftime("%Y-%m-%dT%H:%M:%SZ")

    url = f"https://api.twitch.tv/helix/videos?user_id={id_canal}&type=archive&started_at={fecha_inicio_str}&first=100"

    headers = {
        "Client-ID": client_id,
        "Authorization": f"Bearer {oauth_token}"
    }

    vods_data = []

    while url:
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json()
            vods = data["data"]

            for vod in vods:
                vod_id = int(vod["id"])
                fecha_inicio = datetime.strptime(vod["created_at"], "%Y-%m-%dT%H:%M:%SZ")
                fecha_final = datetime.strptime(vod["published_at"], "%Y-%m-%dT%H:%M:%SZ")
                duracion = vod["duration"]
                visitas = int(vod["view_count"])
                nombre = vod["title"].lower().replace(' ', '_')
                descripcion = vod["description"]
                url_vod = f"https://www.twitch.tv/videos/{vod_id}"

                vods_data.append({
                    "ID del VOD": vod_id,
                    "URL del VOD": url_vod,
                    "Nombre del VOD": nombre,
                    "Duración": duracion,
                    "Fecha de inicio": fecha_inicio,
                    "Fecha de finalización": fecha_final,
                    "Visitas": visitas,
                    
                })

            pagination = data.get("pagination", None)
            if pagination and "cursor" in pagination:
                cursor = pagination["cursor"]
                url = f"https://api.twitch.tv/helix/videos?user_id={id_canal}&type=archive&started_at={fecha_inicio_str}&first=100&after={cursor}"
            else:
                url = None

        else:
            print("Se produjo un error al realizar la solicitud GET.")
            return None

    # Crear un DataFrame a partir de los datos
    df_vods = pd.DataFrame(vods_data)

    return df_vods

df = obtener_vods(id_canal)
df.info(memory_usage='deep')
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID del VOD             37 non-null     int64         
 1   URL del VOD            37 non-null     object        
 2   Nombre del VOD         37 non-null     object        
 3   Duración               37 non-null     object        
 4   Fecha de inicio        37 non-null     datetime64[ns]
 5   Fecha de finalización  37 non-null     datetime64[ns]
 6   Visitas                37 non-null     int64         
dtypes: datetime64[ns](2), int64(2), object(3)
memory usage: 15.8 KB


,ID del VOD,URL del VOD,Nombre del VOD,Duración,Fecha de inicio,Fecha de finalización,Visitas
0,1891232780,https://www.twitch.tv/videos/1891232780,"humillando_a_ari,_rivers_y_el_mariana_/_centra...",2h56m54s,2023-08-06 00:00:14,2023-08-06 00:00:14,326716
1,1890082352,https://www.twitch.tv/videos/1890082352,tengo_a_los_míos_hablando_italiano_de_lunes_a_...,4h4m4s,2023-08-04 18:18:04,2023-08-04 18:18:04,335459
2,1889304904,https://www.twitch.tv/videos/1889304904,"caos_caos_caos_/_trailer_""el_dios_de_todo""_/_e...",6h55m42s,2023-08-03 20:07:06,2023-08-03 20:07:06,596488
3,1888331436,https://www.twitch.tv/videos/1888331436,"pláticamos_+_trailer_""el_dios_de_todo""_+_juegu...",3h27m35s,2023-08-02 18:00:25,2023-08-02 18:00:25,341156
4,1884786123,https://www.twitch.tv/videos/1884786123,final_four_/_semifinal_aniquiladores_fc_🆚_el_b...,1h43m44s,2023-07-29 14:28:33,2023-07-29 14:28:33,525443


In [9]:
df.to_csv('twitch_api_juansguarnizo_csv.csv', index=False)